In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
num_data = 100000

In [ ]:
%pip uninstall -y codeflare-sdk

In [ ]:
%pip install 'feast[postgres, redis]' psycopg2

In [ ]:
from numpy import arange, random, round, exp, array
from numpy.random import seed
from datetime import datetime, timedelta


seed(123456)
images = arange(0, num_data)

other_bottom = random.rand(num_data) * 640
other_left = random.rand(num_data) * 640
other_right = 640 - (640 - other_left) * random.rand(num_data)
other_top = 640 - (640 - other_bottom) * random.rand(num_data)

other_speed = abs(random.normal(40, 30, num_data))
your_speed = abs(random.normal(40, 30, num_data))


def logistic_function(x):
    """
    Calculates the logistic (sigmoid) function for a given input x.

    Args:
        x (float or np.ndarray): The input value(s).

    Returns:
        float or np.ndarray: The output of the logistic function,
                             ranging between 0 and 1.
    """
    return 2 / (1 + exp(-x))

size = (other_top - other_bottom) * (other_right - other_left)
distance = (640**2) / size - 1
relative_speed = other_speed - your_speed
relative_pressure = relative_speed - your_speed

brake_amount = array([logistic_function(relative_pressure[i] / distance[i]) if distance[i] < 1000 and relative_pressure[i] < 0 else 0. for i in range(num_data)]) + 0.05 * (1 - 2 * random.rand(num_data))
brake_amount = array([(1 if x > 1 else (0 if x < 0 else x)) for x in brake_amount])

timestamps = [datetime.now() - timedelta(minutes=5 * i) + timedelta(days=30) for i in range(num_data)]

In [ ]:
import matplotlib.pyplot as plt

ax = plt.gca() # Get the current Axes
print(ax)

ax.hist(brake_amount, bins=20)

In [ ]:
from pandas import DataFrame

data = DataFrame()
data["image_id"] = images
data["other_bottom"] = other_bottom
data["other_left"] = other_left
data["other_right"] = other_right
data["other_top"] = other_top
data["other_speed"] = other_speed
data["your_speed"] = your_speed
data["brake_amount"] = brake_amount
data["event_timestamp"] = timestamps
data["created"] = timestamps

In [ ]:
from os import getenv
from sqlalchemy import create_engine, engine

connection_string = engine.URL.create(
    drivername="postgresql",
    username=getenv('USER'),
    password=getenv('PASSWORD'),
    host=getenv('HOST'),
    database=getenv('DBNAME'),
)

this_engine = create_engine(connection_string)
data.to_sql('approaching_vehicle', schema="distance", con=this_engine, if_exists="append", index=False)

In [ ]:
from psycopg2 import connect
from os import getenv

try:
    conn = connect(dbname=getenv('DBNAME'), user=getenv('USER'), host=getenv('HOST'), password=getenv('PASSWORD'))
except Exception as e:
    print("I am unable to connect to the database")
    print(e)

with conn.cursor() as curs:
    curs.execute("SELECT table_name FROM INFORMATION_SCHEMA.TABLES")
    single_row = curs.fetchone()
    print(single_row)

conn.close()

In [ ]:
try:
    conn = connect(dbname=getenv('DBNAME'), user=getenv('USER'), host=getenv('HOST'), password=getenv('PASSWORD'))
except Exception as e:
    print("I am unable to connect to the database")
    print(e)

with conn.cursor() as curs:
    curs.execute("SELECT * from distance.approaching_vehicle")
    colnames = [desc[0] for desc in curs.description]
    print(colnames)
conn.close()